In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matlab.engine
import socket, struct
import threading
import concurrent.futures
import random

2024-04-24 14:23:27.291396: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 14:23:27.293796: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 14:23:27.322114: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 14:23:27.811455: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
## matlab api connection
eng = matlab.engine.start_matlab()
eng.cd(r'/home/pvm8318/Documents/Reinforcement/2023b')
eng.addpath(r'/home/pvm8318/Documents/Reinforcement/2023b')
def SimRun():
    eng.sim('Buck_Converter.slx')
    return

In [3]:
## TCP Connection
MESSAGE_SIZE = 24
DELIMITER = b'\n'
TCP_IP = '127.0.0.1'
TCP_PORT = 50000
BUFFER_SIZE = MESSAGE_SIZE if MESSAGE_SIZE else 32  # Minimum for two doubles


def send_data(conn, val):
    """Sends two double-precision numbers."""
    # Fixed Size
    msg = struct.pack('>d', val)
    conn.send(msg)

def receive_data(conn):
    """Receives three double-precision numbers."""
    if MESSAGE_SIZE:
        data = conn.recv(MESSAGE_SIZE)
        val1, val2, Time = struct.unpack('>ddd', data)
    else:
        # Delimiter
        val1 = None
        val2 = None
        Time = None
        while True:
            data = conn.recv(BUFFER_SIZE)
            if DELIMITER in data:
                val1_bytes, remaining = data.split(DELIMITER, 1)
                val1 = struct.unpack('>d', val1_bytes)[0]
                if DELIMITER in remaining:
                    val2_bytes, time_bytes = remaining.split(DELIMITER, 1)
                    val2 = struct.unpack('>d', val2_bytes)[0]
                    Time = struct.unpack('>d', time_bytes)[0]
                    break
    return val1, val2, Time

# Close the existing socket connection if it is open

In [4]:
## Buck converter parameters 
Vref = 5
u = 0
R = 1.0  # Resistance
L = 0.1  # Inductance
C = 1e-3  # Capacitance
Vin = 12.0  # Input voltage
Vref = 5.0  # Reference output voltage.0
# State-space representation of the buck converter
A = np.array([[0, 1 / C], [-1 / L, -R / L]])
B = np.array([[0], [1 / L]])
#steady state calculation
duty_cycle =Vref/Vin
Iout = Vref/R
ILref = Iout/duty_cycle

In [5]:
def websocket ():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((TCP_IP, TCP_PORT))
    print('Waiting for Simulink to start')
    s.listen(1)
    conn, addr = s.accept()
    return conn

In [11]:
# Define the actor and critic networks
class Actor(tf.keras.Model):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.fc1 = Dense(64, activation='relu')
        self.fc2 = Dense(64, activation='relu')
        self.output_layer = Dense(action_dim, activation='sigmoid')
    def call(self, state):
        x = self.fc1(state)
        x = self.fc2(x)
        action = self.output_layer(x) * max_action
        return action

class Critic(tf.keras.Model):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.fc1 = Dense(64, activation='relu')
        self.fc2 = Dense(64, activation='relu')
        self.output_layer = Dense(1)

    def call(self, state, action):
        x = self.fc1(state)
        action = tf.squeeze(action, axis=[1,2])  # remove dimensions of size 1
        x = tf.concat([x, action], axis=-1)
        x = self.fc2(x)
        q_value = self.output_layer(x)

def rewardcal(x, u):
    V = x[0]
    IL = x[1]
    Q = 10*np.eye(2)  # State penalty matrix
    R = 1 
    reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 
    # reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 - u**2 * R
    return reward


def isdone(x, t):
    # Define the desirable band
    desirable_band = [4.8, 5.2]

    # Initialize the start time and t0
    t0 = None

    V = x[0]
    IL = x[1]
    
    # Check if the state is within the desirable band
    if V >= desirable_band[0] and V <= desirable_band[1]:
        # Check if t0 is None (first time in the band)
        if t0 is None:
            t0 = t
        # Check if the state has been within the desirable band for 0.5 seconds
        elif t - t0 >= 0.5:
            return True
    else:
        # Reset t0 if V gets out of the band
        t0 = None
    
    return False

# Initialize environment and hyperparameters
state_dim = 2  # I have Voltage and Current that describes the state of the system
action_dim = 1  # Duty cycle
max_action = 1.0  # Maximum duty cycle value
actor_lr = 0.001
critic_lr = 0.002
gamma = 0.99  # Discount factor
tau = 0.005  # Target network update rate

actor = Actor(state_dim, action_dim, max_action)
actor_target = Actor(state_dim, action_dim, max_action)
actor_target.set_weights(actor.get_weights())

critic = Critic(state_dim, action_dim)
critic_target = Critic(state_dim, action_dim)
critic_target.set_weights(critic.get_weights())

actor_optimizer = Adam(learning_rate=actor_lr)
critic_optimizer = Adam(learning_rate=critic_lr)

# Define the replay buffer (store experiences)
replay_buffer = []
num_episodes = 100
Vinit = 0
Iinit = 0
max_steps = 30/1e-5  # Maximum number of steps per episode
batch_size = 32  # Replace with your desired batch size

# Training loop


# After training, use the trained actor network to control the buck converter
# You can query the actor network with the current state to get the optimal duty cycle
# Remember to adapt this code to your specific Simulink model and requirements


In [13]:
conn.close()

In [ ]:
for episode in range(num_episodes):
    t1 = threading.Thread(target=SimRun)
    t1.start()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future2 = executor.submit(websocket)
        conn = future2.result()
    time = 0
    state = np.array([Vinit,Iinit])  # Initial state from Simulink
    total_reward = 0

    while time < 30:
        # Choose action using actor network
        u = action[0][0]
        send_data(conn, u)
        val1, val2,time = receive_data(conn)
        next_state = np.array([val1, val2])
        reward = rewardcal(next_state, u)
        done = isdone(next_state, time)
        action = actor(np.expand_dims(state, axis=0))
        replay_buffer.append((state, action, reward, next_state, done))

        # Update critic network
        if len(replay_buffer) < batch_size:
            continue
        batch = random.sample(replay_buffer, batch_size)
        states = np.array([x[0] for x in batch])
        actions = np.array([x[1] for x in batch])
        rewards = np.array([x[2] for x in batch])
        next_states = np.array([x[3] for x in batch])
        dones = np.array([x[4] for x in batch])
        next_actions = actor_target(next_states)
        target_q_values = critic_target(next_states, next_actions)
        target_q_values = rewards + gamma * target_q_values * (1 - dones)
        with tf.GradientTape() as tape:
            q_values = critic(states, actions)
            critic_loss = tf.reduce_mean(tf.square(target_q_values - q_values))
        critic_gradients = tape.gradient(critic_loss, critic.trainable_variables)
        # Store experience in replay buffer
        if len(replay_buffer) >= batch_size:
            batch = random.sample(replay_buffer, batch_size)
        else:
            continue
        critic_optimizer.apply_gradients(zip(critic_gradients, critic.trainable_variables))

        # Update actor network
        with tf.GradientTape() as tape:
            new_actions = actor(states)
            actor_loss = -tf.reduce_mean(critic(states, new_actions))
        actor_gradients = tape.gradient(actor_loss, actor.trainable_variables)
        actor_optimizer.apply_gradients(zip(actor_gradients, actor.trainable_variables))

        # Update target networks
        for target, source in zip(actor_target.trainable_variables, actor.trainable_variables):
            target.assign(target * (1 - tau) + source * tau)
        for target, source in zip(critic_target.trainable_variables, critic.trainable_variables):
            target.assign(target * (1 - tau) + source * tau)

        state = next_state
        total_reward += reward
        print(f"Episode {episode + 1}: Total Reward = {total_reward:.2f}")
        print('Duty cycle is:', u)
        print('time is:', time)
        if done:
            break
    conn.close()
    
    

In [34]:
conn.close()

In [35]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input

# Define the shared network architecture
def shared_network(state_dim, action_dim):
    # Input layer
    inputs = Input(shape=(state_dim,))

    # Shared hidden layers
    hidden1 = Dense(64, activation='relu')(inputs)
    hidden2 = Dense(64, activation='relu')(hidden1)

    # Actor head (output for action)
    actor_output = Dense(action_dim, activation='sigmoid')(hidden2)  # Output range: [0, 1]

    # Critic head (output for state value)
    critic_output = Dense(1)(hidden2)  # Linear activation for value estimation

    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=[actor_output, critic_output])
    return model

# Example usage
state_dim = 2  # Number of state features (Vc and IL)
action_dim = 1  # Single action (duty cycle)

# Create the shared network
shared_net = shared_network(state_dim, action_dim)

# Print the model summary
shared_net.summary()


# Initialize Actor and Critic networks
actor = Actor(state_dim, action_dim, max_action)
critic = Critic(state_dim, action_dim)

# Initialize target networks
target_actor = Actor(state_dim, action_dim, max_action)
target_critic = Critic(state_dim, action_dim)

# Copy weights from original networks to target networks
target_actor.set_weights(actor.get_weights())
target_critic.set_weights(critic.get_weights())

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define loss function
loss_function = tf.keras.losses.MeanSquaredError()

# Define replay buffer
replay_buffer = []

# Define noise process
noise_process = np.random.normal(0, 0.1, size=(action_dim,))

# Define training loop
for episode in range(1000):  # number of episodes
    # Reset the environment and get the initial state
    t1 = threading.Thread(target=SimRun)
    t1.start()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future2 = executor.submit(websocket)
        conn = future2.result()
    time = 0
    state = np.array([Vinit,Iinit])  # Initial state from Simulink
    total_reward = 0
    u=0
    while time<30:  # maximum steps per episode
        # Select action
        send_data(conn, u)
        val1, val2,time = receive_data(conn)
        next_state = np.array([val1, val2])
        reward = rewardcal(next_state, u)
        done = isdone(next_state, time)
        action = actor(np.expand_dims(state, axis=0))
        replay_buffer.append((state, action, reward, next_state, done))

        # Sample a random minibatch of N transitions from the replay buffer
        if len(replay_buffer) >= 64:
            batch = random.sample(replay_buffer, 64)
        else:
            batch = []

        # Update Critic network
        with tf.GradientTape() as tape:
            q_values = critic(state, tf.expand_dims(action, axis=0))  # Reshape action tensor
            critic_loss = loss_function(target_q_values, q_values)
        critic_gradients = tape.gradient(critic_loss, critic.trainable_variables)
        optimizer.apply_gradients(zip(critic_gradients, critic.trainable_variables))

        # Update Actor network
        with tf.GradientTape() as tapeimport tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np

class Actor(tf.keras.Model):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.fc1 = Dense(64, activation='relu')
        self.fc2 = Dense(64, activation='relu')
        self.output_layer = Dense(action_dim, activation='tanh')

        self.max_action = max_action

    def call(self, state):
        x = self.fc1(state)
        x = self.fc2(x)
        x = self.output_layer(x)
        return self.max_action * x

class Critic(tf.keras.Model):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.fc1 = Dense(64, activation='relu')
        self.fc2 = Dense(64, activation='relu')
        self.output_layer = Dense(1)

    def call(self, state, action):
        x = tf.concat([state, action], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.output_layer(x)
actor_gradients, actor.trainable_variables))

        # Update target networks
        target_actor.set_weights(target_actor.get_weights() * 0.995 + actor.get_weights() * 0.005)
        target_critic.set_weights(target_critic.get_weights() * 0.995 + critic.get_weights() * 0.005)

        # Update state and total reward
        state = next_state
        total_reward += reward
        
        print(f"Episode {episode + 1}: Total Reward = {total_reward:.2f}")
        print('Duty cycle is:', u)
        print('time is:', time)

        if done:
            break
    conn.close()

Waiting for Simulink to start


2024-04-24 13:14:40.890900: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: ConcatOp : Ranks of all input tensors should match: shape[0] = [2] vs. shape[1] = [1,1,1]


InvalidArgumentError: Exception encountered when calling Critic.call().

[1m{{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [2] vs. shape[1] = [1,1,1] [Op:ConcatV2] name: concat[0m

Arguments received by Critic.call():
  • state=tf.Tensor(shape=(2,), dtype=float32)
  • action=tf.Tensor(shape=(1, 1, 1), dtype=float32)

MATLAB System block 'Buck_Converter/TCP//IP Receive' error occurred when invoking 'stepImpl' method of 'instrument.system.TCPIPReceive'. The error was thrown from '
 <a href="matlab:opentoline('/usr/local/MATLAB/R2023b/toolbox/instrument/instrumentblks/+instrument/+system/TCPIPReceive.m',388, 0)">'/usr/local/MATLAB/R2023b/toolbox/instrument/instrumentblks/+instrument/+system/TCPIPReceive.m'</a> at line 388'.
Caused by:
    Error receiving data from the remote server.
    Additional Information: Operation timed out before requested data was received.

Exception in thread Thread-26 (SimRun):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/pvm8318/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_318872/1695

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
import numpy as np

# Define the shared network architecture
def shared_network(state_dim, action_dim):
    inputs = Input(shape=(state_dim,))
    hidden1 = Dense(64, activation='relu')(inputs)
    hidden2 = Dense(64, activation='relu')(hidden1)
    actor_output = Dense(action_dim, activation='sigmoid')(hidden2)  # Output range: [0, 1]
    critic_output = Dense(1)(hidden2)  # Linear activation for value estimation
    model = tf.keras.Model(inputs=inputs, outputs=[actor_output, critic_output])
    return model

# Example usage
state_dim = 2  # Number of state features (Vc and IL)
action_dim = 1  # Single action (duty cycle)
Vinit = 0
Iinit = 0
max_steps = 30/1e-5 
# Create the shared network
shared_net = shared_network(state_dim, action_dim)

# Define optimizer and loss functions
actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Actor loss function (policy gradient with advantage)
def actor_loss(log_probs, advantages):
    return -tf.reduce_mean(log_probs * advantages)

# Critic loss function (mean squared error)
def critic_loss(targets, predicted_values):
    return tf.reduce_mean(tf.square(targets - predicted_values))

# Compute advantages for a specific state-action pair
def compute_advantage(q_value, v_value):
    return q_value - v_value

# Training loop
num_episodes = 1000  # Number of episodes
runtime = 30  # Maximum steps per episode
gamma = 0.9  # Discount factor

for episode in range(num_episodes):
    t1 = threading.Thread(target=SimRun)
    t1.start()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future2 = executor.submit(websocket)
        conn = future2.result()
    # Reset the environment and get the initial state
    state = np.array([Vinit, Iinit])  # Replace with actual initial state
    total_reward = 0
    time = 0
    u=0
    while time < runtime:
        
        action = shared_net(np.expand_dims(state, axis=0))[0][0] 
        u=action
        send_data(conn, u)
        val1, val2,time = receive_data(conn)
        next_state = np.array([val1, val2])
        reward = rewardcal(next_state, u)
        done = isdone(next_state, time)

        # Compute the advantage for this state-action pair
        q_value = shared_net(np.expand_dims(state, axis=0))[1][0]
        v_value = shared_net(np.expand_dims(state, axis=0))[1][0]
        advantage = compute_advantage(q_value, v_value)

        # Update the Actor network
        with tf.GradientTape() as tape:
            log_prob = tf.math.log(action)  # Assuming continuous action space
            actor_loss_value = actor_loss(log_prob, advantage)
        actor_gradients = tape.gradient(actor_loss_value, shared_net.trainable_variables)
        actor_optimizer.apply_gradients(zip(actor_gradients, shared_net.trainable_variables))

        # Update the Critic network
        with tf.GradientTape() as tape:
            target_value = total_reward  # Compute the return for the trajectory
            critic_loss_value = critic_loss(target_value, shared_net(np.expand_dims(state, axis=0))[1][0])
        critic_gradients = tape.gradient(critic_loss_value, shared_net.trainable_variables)
        critic_optimizer.apply_gradients(zip(critic_gradients, shared_net.trainable_variables))

        # Update state for the next step
        state = next_state
        total_reward += reward
        # Print the total reward for the episode
        print("reward is:", reward)
        print('Duty cycle is:', u)
        print('time is:', time)

        if done:
            break  # End the episode if termination condition is met

    print(f"Episode {episode + 1}: Total Reward = {total_reward:.2f}")

Waiting for Simulink to start


NameError: name 'rewardcal' is not defined

MATLAB System block 'Buck_Converter/TCP//IP Receive' error occurred when invoking 'stepImpl' method of 'instrument.system.TCPIPReceive'. The error was thrown from '
 <a href="matlab:opentoline('/usr/local/MATLAB/R2023b/toolbox/instrument/instrumentblks/+instrument/+system/TCPIPReceive.m',388, 0)">'/usr/local/MATLAB/R2023b/toolbox/instrument/instrumentblks/+instrument/+system/TCPIPReceive.m'</a> at line 388'.
Caused by:
    Error receiving data from the remote server.
    Additional Information: Operation timed out before requested data was received.

Exception in thread Thread-6 (SimRun):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/pvm8318/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_436457/16952